In [24]:
import pandas as pd
import numpy as np

In [25]:
# Import Graphical Plotting Liabraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
# Import Linear Regrssion Machine Learning Liabraries
from sklearn import preprocessing
# from sklearn.preprocessing import ploynomial features
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , Ridge, Lasso
from sklearn.metrics import r2_score

In [27]:
data = pd.read_csv("/Users/manishuxuid/Desktop/FSDS/FSDS Notes/25 Sep- Lasso & Ridge Regression/car-mpg.csv")
data.head()

,mpg,cyl,disp,hp,wt,acc,yr,origin,car_type,car_name
0,18.0,8,307.0,130,3504,12.0,70,1,0,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,0,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,0,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,0,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,0,ford torino


In [28]:
# Drop car name
# Replace origin into 1, 2, 3 .. dont forget get_dummies
# Replace ? with nan
# Replace all nam with median

data = data.drop(['car_name'], axis = 1)
data['origin'] = data['origin'].replace({1 : 'america' , 2 : 'europe', 3 : 'asia'})
data = pd.get_dummies(data, columns = ['origin'],dtype = int)
data = data.replace('?', np.nan)
# data = data.apply(Lamda x : x.fillna(x.median()), axis = 0)




In [29]:
data = data.apply(pd.to_numeric,errors = 'ignore')
#  Fill missing values with median only for numeric columns
numeric_cols = data.select_dtypes(include = [np.number]).columns
data[numeric_cols] = data[numeric_cols].apply(lambda x : x.fillna(x.median()))

/var/folders/w8/5124_l8n1md8s19brt04j5mr0000gn/T/ipykernel_32982/3221036309.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric,errors = 'ignore')


In [30]:
data.head()

,mpg,cyl,disp,hp,wt,acc,yr,car_type,origin_america,origin_asia,origin_europe
0,18.0,8,307.0,130.0,3504,12.0,70,0,1,0,0
1,15.0,8,350.0,165.0,3693,11.5,70,0,1,0,0
2,18.0,8,318.0,150.0,3436,11.0,70,0,1,0,0
3,16.0,8,304.0,150.0,3433,12.0,70,0,1,0,0
4,17.0,8,302.0,140.0,3449,10.5,70,0,1,0,0


# Model Building

In [32]:
x = data.drop(['mpg'], axis = 1) #independent variable
y = data[['mpg']] # dependent variable

In [35]:
# Scaling the Data
x_s = preprocessing.scale(x)
x_s = pd.DataFrame(x_s, columns = x.columns) #converting scaled data into dataframe
y_s = preprocessing.scale(y)
y_s = pd.DataFrame(y_s, columns = y.columns) # ideally train_test data should be in columns

In [36]:
x_s

,cyl,disp,hp,wt,acc,yr,car_type,origin_america,origin_asia,origin_europe
0,1.498191,1.090604,0.673118,0.630870,-1.295498,-1.627426,-1.062235,0.773559,-0.497643,-0.461968
1,1.498191,1.503514,1.589958,0.854333,-1.477038,-1.627426,-1.062235,0.773559,-0.497643,-0.461968
2,1.498191,1.196232,1.197027,0.550470,-1.658577,-1.627426,-1.062235,0.773559,-0.497643,-0.461968
3,1.498191,1.061796,1.197027,0.546923,-1.295498,-1.627426,-1.062235,0.773559,-0.497643,-0.461968
4,1.498191,1.042591,0.935072,0.565841,-1.840117,-1.627426,-1.062235,0.773559,-0.497643,-0.461968
...,...,...,...,...,...,...,...,...,...,...
393,-0.856321,-0.513026,-0.479482,-0.213324,0.011586,1.621983,0.941412,0.773559,-0.497643,-0.461968
394,-0.856321,-0.925936,-1.370127,-0.993671,3.279296,1.621983,0.941412,-1.292726,-0.497643,2.164651
395,-0.856321,-0.561039,-0.531873,-0.798585,-1.440730,1.621983,0.941412,0.773559,-0.497643,-0.461968
396,-0.856321,-0.705077,-0.662850,-0.408411,1.100822,1.621983,0.941412,0.773559,-0.497643,-0.461968


In [37]:
y_s

,mpg
0,-0.706439
1,-1.090751
2,-0.706439
3,-0.962647
4,-0.834543
...,...
393,0.446497
394,2.624265
395,1.087017
396,0.574601


In [40]:
# Split into train, test set
x_train, x_test, y_train, y_test = train_test_split(x_s, y_s, test_size = 0.20, random_state = 0)
x_train.shape

(318, 10)

# Simple Linear Model

In [43]:
# Fit simple linear models and fit coefficiants
regression_model = LinearRegression()
regression_model.fit(x_train, y_train)

for idx, col_name in  enumerate(x_train.columns):
    print("The coefficiant forc {} is {}". format(col_name, regression_model.coef_[0][idx]))
intercept = regression_model.intercept_[0]
print("The intercept is {}". format (intercept))

The coefficiant forc cyl is 0.2463877605357169
The coefficiant forc disp is 0.29177092098664537
The coefficiant forc hp is -0.18081621820393656
The coefficiant forc wt is -0.6675530609868129
The coefficiant forc acc is 0.06537309205777056
The coefficiant forc yr is 0.3481770259426719
The coefficiant forc car_type is 0.3339231253960364
The coefficiant forc origin_america is -0.08117984631927017
The coefficiant forc origin_asia is 0.06986098209664901
The coefficiant forc origin_europe is 0.030003161242288263
The intercept is -0.018006831370923227


# regularize ridge regression

In [46]:
# alpha factor here is lambda(penalty term) which helps to reduce the magnificient og coefficient
ridge_model = Ridge(alpha = 0.4)
ridge_model.fit(x_train, y_train)

print("Ridge model coef: {}".format(ridge_model.coef_))
 # as the data has 10 columns hence 10 coefficient here     


Ridge model coef: [ 0.24242411  0.28008024 -0.18071842 -0.65711583  0.06353256  0.34721777
  0.32998816 -0.08077573  0.06989674  0.02945199]


# Regularized Lasso Regression

In [48]:
# alpha factor here is lambda(penalty term) which helps to reduce the magnificient og coefficient
lasso_model = Lasso(alpha = 0.1)
lasso_model.fit(x_train, y_train)

print("Lasso model coef: {}".format(lasso_model.coef_))
 # as the data has 10 columns hence 10 coefficient here    

Lasso model coef: [-0.         -0.         -0.07247557 -0.45867691  0.          0.2698134
  0.11341188 -0.04988145  0.          0.        ]


# Score Comparision